# Prompt Engineering Lifecycle

### Setup

In [ ]:
# You can set them inline
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith math496"

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Log a trace

In [7]:
from app import langsmith_rag

question = "How do I set up tracing to LangSmith with @traceable?"
langsmith_rag(question)

"To set up tracing to LangSmith with @traceable, first, ensure that the LANGSMITH_TRACING environment variable is set to 'true' and the LANGSMITH_API_KEY is configured with your API key. Then, simply decorate the desired function with the @traceable decorator in your Python code. Make sure to call any wrapped synchronous functions using the await keyword to correctly log traces."

### Create a Dataset

Let's create a dataset to evaluate this particular step of our application

In [9]:
from langsmith import Client

example_dataset = [
    (
        "What are the best open-world video games?",
        """Open-world games allow players to freely explore large environments and complete missions at their own pace. 
Some popular examples include The Legend of Zelda: Breath of the Wild, Red Dead Redemption 2, and Grand Theft Auto V. 
These games are known for their immersive worlds and freedom of gameplay.""",
        "Some of the best open-world video games are The Legend of Zelda: Breath of the Wild, Red Dead Redemption 2, and GTA V."
    ),
    (
        "Where can I find free PC games?",
        """There are many platforms where you can find free PC games. 
Epic Games Store gives away a free game every week. 
Steam also has a Free to Play section that includes games like Dota 2 and Counter-Strike 2. 
You can also visit itch.io for free indie games made by smaller developers.""",
        "You can find free PC games on Epic Games Store, Steam (Free to Play section), and itch.io."
    ),
    (
        "What are some fun multiplayer games to play with friends?",
        """Multiplayer games allow players to team up or compete with friends online. 
Popular multiplayer titles include Among Us, Fortnite, Valorant, Rocket League, and Minecraft. 
These games are great for having fun and connecting with friends around the world.""",
        "Some fun multiplayer games to play with friends include Among Us, Fortnite, Valorant, Rocket League, and Minecraft."
    ),
]

client = Client()
dataset_name = "Video Game Questions"

# Create dataset
dataset = client.create_dataset(
    dataset_name=dataset_name, description="Simple questions about video games"
)


# Prepare inputs and outputs
inputs = [{"question": q, "context": c} for q, c, _ in example_dataset]
outputs = [{"output": o} for _, _, o in example_dataset]

# Create examples in the dataset
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset.id,
)


{'example_ids': ['71232cb6-71cf-4f82-8558-6a3f0410569d',
  '6f0b0fcb-9e33-41dd-8b50-ca29068378b6',
  'a06ac72a-ded0-4a4c-b8a6-ea9aa7cc29f0'],
 'count': 3}

### Update our Application to use Prompt Hub

We're going to pretty much define the same RAG application as before - with one crucial improvement.

Instead of pulling our `RAG_PROMPT` from utils.py, we're going to connect to the Prompt Hub in LangSmith.

Let's add the code snippet that will pull down our prompt that we just iterated on!

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client

prompt = client.pull_prompt("updated_video")

In [15]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from langsmith.client import convert_prompt_to_openai_format
from openai import OpenAI
from typing import List
import nest_asyncio

MODEL_NAME = "gpt-4o-mini"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0



openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    
    formatted_prompt = prompt.invoke({"context":formatted_docs, "question": question})
    messages = convert_prompt_to_openai_format(formatted_prompt)["messages"]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [21]:
question = "What are the best open-world video games?"
question1 = "Where can I find free PC games?"
question2 = "What are some fun multiplayer games to play with friends?"
langsmith_rag(question)
langsmith_rag(question1)
langsmith_rag(question2)

"Here are some fun multiplayer games to enjoy with friends:\n\n1. **Among Us** - A social deduction game where players work together on a spaceship while trying to identify an impostor among them.\n\n2. **Jackbox Party Packs** - A collection of party games, each pack includes different mini-games that are great for groups, ranging from trivia to drawing challenges.\n\n3. **Minecraft** - A sandbox game that allows players to build, explore, and survive together in a blocky world. Great for creativity and teamwork.\n\n4. **Fortnite** - A battle royale game with building mechanics, where players can team up to fight against others in an ever-shrinking map.\n\n5. **Rocket League** - A unique blend of soccer and cars, where players control rocket-powered vehicles to score goals.\n\n6. **Overcooked! 2** - A chaotic cooking simulator where players must cooperate to prepare and serve meals in increasingly challenging kitchens.\n\n7. **A Way Out** - A co-op adventure game focused on two prisone